# Seizure Predicition Procedure
1.   Combine all EEG signal channels using Averaging
2.   Filter using bass band filters for increasing SNR
3.   Feature extraction
4.    Normalization
5.   SVM Vs. KNN for classification



# Importing and installing needed libraries & setting google drive folder


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from numpy import abs, sum, linspace
from numpy.fft import rfft
import librosa, librosa.display

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

#Ready surrogate channel( CSV files containing averaged output)

In [ ]:
df1= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df1.csv')
df1_np= df1['0'].to_numpy()

df2= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df2.csv')
df2_np= df2['0'].to_numpy()

df3= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df3.csv')
df3_np= df3['0'].to_numpy()

df4= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df4.csv')
df4_np= df4['0'].to_numpy()

df5= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df5.csv')
df5_np= df5['0'].to_numpy()

df6= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df6.csv')
df6_np= df6['0'].to_numpy()

df7= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df7.csv')
df7_np= df7['0'].to_numpy()

df8= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df8.csv')
df8_np= df8['0'].to_numpy()

df9= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df9.csv')
df9_np= df9['0'].to_numpy()

df10= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df10.csv')
df10_np= df10['0'].to_numpy()

df11= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df11.csv')
df11_np= df11['0'].to_numpy()

df12= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df12.csv')
df12_np= df12['0'].to_numpy()

df13= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df13.csv')
df13_np= df13['0'].to_numpy()

df14= pd.read_csv('/content/drive/My Drive/Graduationnn/trial/df14.csv')
df14_np= df14['0'][210:].to_numpy()

#Filtering

In [ ]:
from scipy.signal import butter, lfilter

def butter_bandpass(lowcut, highcut, fs, order=4):
    nyq = 0.5 * fs
    low = lowcut / nyq
    high = highcut / nyq
    b, a = butter(order, [low, high], btype='band')
    return b, a


def butter_bandpass_filter(data, lowcut, highcut, fs, order=4):
    b, a = butter_bandpass(lowcut, highcut, fs, order=order)
    y = lfilter(b, a, data)
    return y

#Featurs Extraction

In [ ]:
#Statistical
def statisticalMean(signal):
  return abs(np.mean(signal))

def statisticalSTD(signal):
  return np.std(signal)

def statisticalSkewness(signal):
  dataframe = pd.DataFrame(signal)
  return dataframe.skew()[0]

def statisticalKurtosis(signal):
  dataframe = pd.DataFrame(signal)
  return dataframe.kurtosis()[0]

In [ ]:
#Spectral
def spectralCentroid(signal):
  #It is calculated as the weighted mean of the frequencies present in the signal, 
  #determined using a Fourier transform, with their magnitudes as the weights.
  n = len(signal)
  timestep = 1/256
  spectrum = abs(np.fft.rfft(signal)) #using FT to compute frequcies present in the signal
  normalized_spectrum = spectrum / sum(spectrum)  # like a probability mass function
  normalized_frequencies = np.linspace(0, 1, len(spectrum))
  spectral_centroid = sum(normalized_frequencies * normalized_spectrum)
  freq = np.fft.rfftfreq(n, d=timestep) #get frequncies
  freq_max= freq.max() 
  # Spectral centroid is calculated as a ratio.  Multiply by your maximum frequency bin to get real frequency.
  spectral_centroid= freq_max * spectral_centroid
  return spectral_centroid

def averageEnergy(signal):
  summation = 0
  for i in range(len(signal)):
    summation = summation + ((signal[i])**2)
  avg_energy = summation/ (len(signal))
  return avg_energy

In [ ]:
def signalFilter(data, fs):
  delta = butter_bandpass_filter(data, 0.5, 4, fs, order=4)
  theta = butter_bandpass_filter(data, 4, 8, fs, order=4)
  alpha = butter_bandpass_filter(data, 8, 12, fs, order=4)
  beta = butter_bandpass_filter(data, 12, 25, fs, order=4)
  return delta,theta,alpha,beta

In [ ]:
def divide_and_filter(signal,frame_length,hop_length):
  
  signal= butter_bandpass_filter(signal, lowcut=0.5, highcut=25, fs=256, order=4)
  frames = librosa.util.frame(signal, frame_length=frame_length, hop_length=hop_length)
  
  e_dataframe= pd.DataFrame()
  std_l=[]
  skew_l=[]
  kurtosis_l=[]
  spC_l=[]
  deltaList= []
  thetaList= []
  alphaList= []
  betaList= []
  
  for frame in frames.T:
    delta,theta,alpha,beta = signalFilter(frame, 256)
    deltaList.append(averageEnergy(delta))
    thetaList.append(averageEnergy(theta))
    alphaList.append(averageEnergy(alpha))
    betaList.append(averageEnergy(beta))
    
    stat_std= statisticalSTD(frame)
    std_l.append(stat_std)

    stat_skew= statisticalSkewness(frame)
    skew_l.append(stat_skew)

    stat_kurtosis= statisticalKurtosis(frame)
    kurtosis_l.append(stat_kurtosis)
    
    spec_cetroid= spectralCentroid(frame)
    spC_l.append(spec_cetroid)

  e_dataframe['Standard Deviation']= std_l
  e_dataframe['Skewness']= skew_l
  e_dataframe['Kurtosis']= kurtosis_l
  e_dataframe['Spectral Centroid']= spC_l
  e_dataframe['delta'] = deltaList
  e_dataframe['theta'] = thetaList
  e_dataframe['alpha'] = alphaList
  e_dataframe['beta'] = betaList

  return e_dataframe

#Normalization (feature Scaling)

In [ ]:
from sklearn.preprocessing import MinMaxScaler
def normalizeData(dataFrame):
  minRange = 0
  maxRange = 100
  scaler = MinMaxScaler(feature_range=(minRange, maxRange), copy=True)
  df = pd.DataFrame(scaler.fit_transform(dataFrame),columns=dataFrame.columns, index=dataFrame.index) 
  return df

#Testing

In [ ]:

df1_ex= divide_and_filter(df1_np,2560,2560)
df1_ex_norm= normalizeData(df1_ex)

df2_ex= divide_and_filter(df2_np,2560,2560)
df2_ex_norm= normalizeData(df2_ex)

df3_ex= divide_and_filter(df3_np,2560,2560)
df3_ex_norm= normalizeData(df3_ex)

df4_ex= divide_and_filter(df4_np,2560,2560)
df4_ex_norm= normalizeData(df4_ex)

df5_ex= divide_and_filter(df5_np,2560,2560)
df5_ex_norm= normalizeData(df5_ex)

df6_ex= divide_and_filter(df6_np,2560,2560)
df6_ex_norm= normalizeData(df6_ex)

df7_ex= divide_and_filter(df7_np,2560,2560)
df7_ex_norm= normalizeData(df7_ex)

df8_ex= divide_and_filter(df8_np,2560,2560)
df8_ex_norm= normalizeData(df8_ex)

df9_ex= divide_and_filter(df9_np,2560,2560)
df9_ex_norm= normalizeData(df9_ex)

df10_ex= divide_and_filter(df10_np,2560,2560)
df10_ex_norm= normalizeData(df10_ex)

df11_ex= divide_and_filter(df11_np,2560,2560)
df11_ex_norm= normalizeData(df11_ex)

df12_ex= divide_and_filter(df12_np,2560,2560)
df12_ex_norm= normalizeData(df12_ex)

df13_ex= divide_and_filter(df13_np,2560,2560)
df13_ex_norm= normalizeData(df13_ex)

df14_ex= divide_and_filter(df14_np,2560,2560)
df14_ex_norm= normalizeData(df14_ex)

In [ ]:
df1_ex_norm["Label"] = 0
df1_ex_norm.iloc[300:303]['Label']=1

df2_ex_norm["Label"] = 0
df2_ex_norm.iloc[146:151]['Label']=1

df3_ex_norm["Label"] = 0
df3_ex_norm.iloc[173:177]['Label']=1

df4_ex_norm["Label"] = 0
df4_ex_norm.iloc[100:106]['Label']=1

df5_ex_norm["Label"] = 0
df5_ex_norm.iloc[172:181]['Label']=1

df6_ex_norm["Label"] = 0
df6_ex_norm.iloc[33:42]['Label']=1

df7_ex_norm["Label"] = 0
df7_ex_norm.iloc[73:80]['Label']=1

df8_ex_norm["Label"] = 0
df8_ex_norm.iloc[308:311]['Label']=1

df9_ex_norm["Label"] = 0
df9_ex_norm.iloc[46:54]['Label']=1

df10_ex_norm["Label"] = 0
df10_ex_norm.iloc[111:120]['Label']=1

df11_ex_norm["Label"] = 0
df11_ex_norm.iloc[235:241]['Label']=1

df12_ex_norm["Label"] = 0
df12_ex_norm.iloc[349:353]['Label']=1

df13_ex_norm["Label"] = 0
df13_ex_norm.iloc[208:213]['Label']=1

df14_ex_norm["Label"] = 0

In [ ]:
frames= [df1_ex_norm[280:320], df2_ex_norm[130:170],df3_ex_norm[150:190],df4_ex_norm[80:120],df5_ex_norm[155:195],
         df6_ex_norm[15:55], df7_ex_norm[55:95],df8_ex_norm[280:320], df9_ex_norm[25:65], df10_ex_norm[95:135], 
         df11_ex_norm[215:255],df12_ex_norm[313:353], df13_ex_norm[176:216], df14_ex_norm[0:20]]
result = pd.concat(frames)
result=result.reset_index()
result=result.drop(columns='index')

In [ ]:
result.head(5)

,Standard Deviation,Skewness,Kurtosis,Spectral Centroid,delta,theta,alpha,beta,Label
0,5.984302,51.877290,19.518346,34.460581,0.919281,4.593203,18.575051,11.474303,0
1,9.379791,46.531777,8.659403,25.335442,1.943445,4.248461,28.115566,39.160667,0
2,13.030767,50.193374,38.259478,32.286168,3.619233,4.276488,21.241020,25.410534,0
3,3.729856,41.554274,8.201615,26.018449,0.509283,2.907127,16.505304,16.966855,0
4,13.971547,53.989292,26.815773,10.311566,4.158961,2.801256,16.613062,31.270486,0


# Support Vector Machine Vs. KNN Classification

#All patients

In [ ]:
#Trial 1: Band bass filter & time and spectral old features
#SVM
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']
# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   
#print(accuracy_model)

new Matrix
[[589   0]
 [  3   5]]
new Matrix
[[568   0]
 [ 13  15]]
new Matrix
[[570   2]
 [  2  22]]
new Matrix
[[584   3]
 [  3   6]]
new Matrix
[[576  11]
 [  3   6]]
the total confusion matrix is [[2887.   16.]
 [  24.   54.]]


In [ ]:
results of svm 
specificity= 0.99
sensitivity= 0.70
accuracy= 0.98

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[589   0]
 [  2   6]]
new Matrix
[[568   0]
 [  6  22]]
new Matrix
[[570   2]
 [  8  16]]
new Matrix
[[579   8]
 [  2   7]]
new Matrix
[[581   6]
 [  1   8]]
the total confusion matrix is [[2887.   16.]
 [  19.   59.]]


In [ ]:
results of svm 
specificity= 0.99
sensitivity= 0.76
accuracy= 0.988

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
# resampling: downsizing
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[95  1]
 [ 2 10]]
new Matrix
[[91  2]
 [ 2 13]]
new Matrix
[[84  5]
 [ 2 17]]
new Matrix
[[83  2]
 [ 7 16]]
new Matrix
[[98  1]
 [ 1  8]]
the total confusion matrix is [[451.  11.]
 [ 14.  64.]]


In [ ]:
results of svm with downsampling
specificity= 0.97
sensitivity= 0.82
accuracy= 0.95

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#knn
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold 
from sklearn.metrics import accuracy_score

classifier = KNeighborsClassifier(n_neighbors=5)
X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=5,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = classifier.fit(X_train, y_train)
    y_predict = classifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   
print(accuracy_model)

new Matrix
[[588   1]
 [  3   5]]
new Matrix
[[568   0]
 [  6  22]]
new Matrix
[[570   2]
 [ 10  14]]
new Matrix
[[582   5]
 [  2   7]]
new Matrix
[[581   6]
 [  1   8]]
the total confusion matrix is [[2889.   14.]
 [  22.   56.]]
[99.32998324958125, 98.99328859060402, 97.98657718120806, 98.8255033557047, 98.8255033557047]


In [ ]:
results of KNN 
specificity= 0.99
sensitivity= 0.72
accuracy= 0.987

#Single Patient

In [ ]:
frames_1= [df1_ex_norm[280:320], df2_ex_norm[130:170],df3_ex_norm[150:190],df4_ex_norm[80:120],df5_ex_norm[155:195],
         df6_ex_norm[15:55]]
result = pd.concat(frames_1)
result=result.reset_index()
result=result.drop(columns='index')


In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=3,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[417   0]
 [  0   3]]
new Matrix
[[401   4]
 [  2  13]]
new Matrix
[[402   0]
 [  3  15]]
the total confusion matrix is [[1220.    4.]
 [   5.   31.]]


In [ ]:
results of SVM for 1 patient
specificity= 0.98
sensitivity= 0.86
accuracy= 0.98

In [ ]:
#Trial 4: Band bass filter & all features without mean and average energy.
#SVM
#resampling= down sampling
from sklearn.svm import SVC
from sklearn.model_selection import KFold 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score, accuracy_score, precision_score, recall_score


svclassifier = SVC(kernel='linear',C=1)

X = result.drop(columns=['Label'],axis=1)
y = result['Label']

# KFold Cross Validation approach
kf = KFold(n_splits=3,shuffle=False)
kf.split(X)    
     
# Initialize the accuracy of the models to blank list. The accuracy of each model will be appended to this list
accuracy_model = []
 
# Iterate over each train-test split
cm_holder=[]
total=np.empty([2, 2])

for train_index, test_index in kf.split(X):
    # Split train-test
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # Train the model
    model = svclassifier.fit(X_train, y_train)
    y_predict = svclassifier.predict(X_test)
    print("new Matrix")
    print(confusion_matrix(y_test,y_predict))
    # Append to accuracy_model the accuracy of the model
    cm_holder.append(confusion_matrix(y_test, y_predict))
    accuracy_model.append(accuracy_score(y_test, model.predict(X_test), normalize=True)*100)
for i in range(len(cm_holder)):
  total= total+ cm_holder[i]
print("the total confusion matrix is", total)   

new Matrix
[[72  0]
 [ 2  6]]
new Matrix
[[64  6]
 [ 0 10]]
new Matrix
[[62  0]
 [ 1 17]]
the total confusion matrix is [[198.   6.]
 [  3.  33.]]


In [ ]:
results of SVM for 1 patient with downsampling
specificity= 0.98
sensitivity= 0.91
accuracy= 0.98